In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

CONNECT_INDY = False
CONNECT_MOBILE = False
USE_SDK = True

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
# add camera geometry
def add_cam(gscene, tool_link="indy0_tcp"):
    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cam", link_name=tool_link,
                       dims=(0.061, 0.061, 0.026), center=(-0.0785, 0, 0.013), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.5), display=True, fixed=True, collision=False)

    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cam_col", link_name=tool_link,
                       dims=(0.081, 0.081, 0.046), center=(-0.0785, 0, 0.013), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.2), display=True, fixed=True, collision=True)

    viewpoint = gscene.create_safe(gtype=GEOTYPE.SPHERE, name="viewpoint", link_name=tool_link,
                                   dims=(0.01, 0.01, 0.01), center=(-0.013, 0, 0), rpy=(0, 0, -np.pi / 2),
                                   color=(1, 0, 0, 0.3), display=True, fixed=True, collision=False, parent="cam")

    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="body", link_name=tool_link,
                       dims=(0.067, 0.067, 0.0335), center=(-0.0785, 0, -0.01675), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 1), display=True, fixed=True, collision=False)

    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="body_col", link_name=tool_link,
                       dims=(0.087, 0.087, 0.0535), center=(-0.0785, 0, -0.01675), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.2), display=True, fixed=True, collision=True)

    gscene.create_safe(gtype=GEOTYPE.SPHERE, name="backhead", link_name=tool_link,
                       dims=(0.067, 0.067, 0.067), center=(-0.0785, 0, -0.0335), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 1), display=True, fixed=True, collision=False)

    gscene.create_safe(gtype=GEOTYPE.SPHERE, name="backhead_col", link_name=tool_link,
                       dims=(0.087, 0.087, 0.087), center=(-0.0785, 0, -0.0335), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.2), display=True, fixed=True, collision=True)
    return viewpoint

In [3]:
INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 1.6
COL_COLOR = (1,1,1,0.2)
    
# if EXP_SCENARIO == ExpType.REMOVE_OBS: ## Obstacle removing
#     BAG_COUNT = 5
#     CLOCK_COUNT = 0
#     TARGET_COUNT = 5
#     LOG_FORCE = False
# else: ## Contact 
#     BAG_COUNT = 3
#     CLOCK_COUNT = 3
#     TARGET_COUNT = 5
#     LOG_FORCE = True

from pkg.controller.combined_robot import *
from pkg.project_config import *

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True} if not USE_SDK else {})

ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

connection command:
kmb0: True
indy1: False


In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene
from pkg.geometry.geometry import GeometryItem
from pkg.geometry.geotype import GEOTYPE

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)

gscene.set_workspace_boundary(
    -1, 4, -2.5, 2.5, -CLEARANCE, WS_HEIGHT, thickness=WALL_THICKNESS)

viewpoint = add_cam(gscene, tool_link="indy1_tcp")

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### MultiICP detector

In [11]:
from pkg.detector.multiICP.multiICP import MultiICP, MultiICP_Obj
from pkg.detector.multiICP.config import *
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

import open3d as o3d
import numpy as np

In [6]:
CONNECT_CAM = True

In [7]:
realsense = RealSense()

if CONNECT_CAM:
    micp = MultiICP(realsense)
    micp.initialize()
else:
    # use manually given camera configs
    config_list, img_dim = load_pickle(RNB_PLANNING_DIR+"release/multiICP_data/cam_configs.pkl")
    micp.initialize(config_list, img_dim)

Start streaming
Initialize Done


### Shared Detector

In [8]:
from pkg.detector.multiICP.shared_detector import SharedDetectorGen
sd = SharedDetectorGen(tuple(reversed(micp.dsize))+(3,))()
sd.init()

### set ICP config

In [12]:
# Load config file of object information
obj_info_dict = get_obj_info()

In [18]:
from pkg.detector.multiICP.heuristics import *

In [25]:
obj_info_dict

{'bed': [],
 'carrier': [],
 'closet': [],
 'cup': [],
 'table': [],
 'tableclock': []}

In [14]:
micp_carrier = MultiICP_Obj(obj_info_dict["carrier"], None,
                        OffsetOnModelCoord("carrier", R=np.identity(3), offset=(0.,0.,0.)))


micp_clock = MultiICP_Obj(obj_info_dict["tableclock"], None,
                        OffsetOnModelCoord("tableclock", R=np.identity(3), offset=(0.,0.,0.)))

In [11]:
# micp_dict = {"suitcase": micp_carrier, "clock": micp_clock}
# micp_dict = {"clock": micp_clock}
micp_dict = {"suitcase": micp_carrier}

In [12]:
# set config information for micp
micp.set_config(micp_dict, sd, crob, viewpoint)

### Detect

In [13]:
pose_dict = micp.detect(visualize=True)

name_mask is None
===== Detected : person, 7 object(s) =====
===== Detected : backpack, 1 object(s) =====
===== Detected : suitcase, 7 object(s) =====
===== Detected : skis, 1 object(s) =====
===== Detected : chair, 6 object(s) =====

'suitcase' is not in gscene. Use manual input for initial guess

Apply point-to-point ICP
registration::RegistrationResult with fitness=8.956772e-01, inlier_rmse=6.408638e-02, and correspondence_set size of 3108
Access transformation to get result.
Transformation is:
[[ 0.97005196  0.24289091  0.00178838 -0.24416827]
 [-0.12963422  0.51147556  0.84946319 -0.28786607]
 [ 0.20541218 -0.82425527  0.52764485  0.99775951]
 [ 0.          0.          0.          1.        ]]
initial: 
[[ 0.97  0.24  0.   -0.24]
 [-0.13  0.51  0.85 -0.29]
 [ 0.21 -0.82  0.53  1.  ]
 [ 0.    0.    0.    1.  ]]
Apply point-to-point ICP
registration::RegistrationResult with fitness=9.803978e-01, inlier_rmse=2.828238e-02, and correspondence_set size of 3401
Access transformation to g